<a href="https://colab.research.google.com/github/jfranco-bah/whisper_testing/blob/main/whisper_tflite_model_generation_and_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install python 3.10 for tensorflow 2.11 compatibility

In [3]:
#The code below installs 3.10 (assuming you now have 3.11) and restarts environment, so you can run your cells.
import sys #for version checker
import os #for restart routine

current_version = "3.11"
desired_version = "3.10"

!export CUR_VER="3.11"
!export DES_VER="3.10"

if desired_version in sys.version:
  print(f'You already have {desired_version}, nothing to install')
elif current_version in sys.version:
  print("Python version is: ", sys.version)

  print("Printing content of /usr/local/lib/python* to see available versions")
  !ls /usr/local/lib/python*

  #install python 3.10 and dev utils
  #you may not need all the dev libraries, but I haven't tested which aren't necessary.
  !sudo apt-get update -y
  !sudo apt-get install -y "python${DES_VER}" "python${DES_VER}-dev" "python${DES_VER}-distutils" "libpython${DES_VER}-dev"
  !sudo apt-get install -y "python${DES_VER}-venv" binfmt-support #recommended in install logs of the command above

  #change alternatives
  !sudo update-alternatives --install /usr/bin/python3 python3 "/usr/bin/python${CUR_VER}" 1 > /dev/null
  !sudo update-alternatives --install /usr/bin/python3 python3 "/usr/bin/python${DES_VER}" 2 > /dev/null

  # install pip
  !curl -sS https://bootstrap.pypa.io/get-pip.py | "python${CUR_VER}"  > /dev/null

  #install colab's dependencies
  !python3 -m pip install ipython traitlets jupyter psutil matplotlib setuptools ipython_genutils ipykernel jupyter_console notebook prompt_toolkit httplib2 astor  > /dev/null

  #minor cleanup
  !sudo apt autoremove > /dev/null

  #link to the old google package
  !ln -s "/usr/local/lib/python${CUR_VER}/dist-packages/google /usr/local/lib/python${DES_VER}/dist-packages/google" > /dev/null

  #this is just to verify if 3.10 folder was indeed created
  print(f"Printing content of /usr/local/lib/python{desired_version}/")
  !ls "/usr/local/lib/python${DES_VER}/"

  !sed -i "s/from IPython.utils import traitlets as _traitlets/import traitlets as _traitlets/" /usr/local/lib/python${DES_VER}/dist-packages/google/colab/*.py
  !sed -i "s/from IPython.utils import traitlets/import traitlets/" /usr/local/lib/python${DES_VER}/dist-packages/google/colab/*.py

  #restart environment so you don't have to do it manually
  # os.kill(os.getpid(), 9)
else:
  print(f"Your out of the box Python is not {desired_version}, so probably the script will not work, so pls feel free to edit the script to ignore then check and re-run: ", sys.version)


You already have 3.10, nothing to install


In [4]:
#check python version
import sys
print("User Current Version:-", sys.version)
!python3 --version
!python --version

User Current Version:- 3.10.12 (main, Jan 17 2025, 14:35:34) [GCC 11.4.0]
Python 3.10.12
Python 3.10.12


##Install TensorFlow, Tranformers and datasets

In [18]:
!pip install tensorflow==2.11.0
!pip install numpy==1.26.4
!pip install librosa soundfile
# !pip install sentence-transformers==2.3.1
!pip install transformers==4.37.2
!pip install datasets

In [19]:
!pip list | grep -E 'tensorflow|transformers|datasets|numpy|pandas'

datasets                     3.2.0
numpy                        1.26.4
pandas                       2.2.3
tensorflow                   2.11.0
tensorflow-estimator         2.11.0
tensorflow-io-gcs-filesystem 0.37.1
transformers                 4.37.2


In [9]:
!pip list

Package                      Version
---------------------------- --------------
absl-py                      2.1.0
aiohappyeyeballs             2.4.4
aiohttp                      3.11.12
aiosignal                    1.3.2
anyio                        4.8.0
argon2-cffi                  23.1.0
argon2-cffi-bindings         21.2.0
arrow                        1.3.0
astor                        0.8.1
asttokens                    3.0.0
astunparse                   1.6.3
async-lru                    2.0.4
async-timeout                5.0.1
attrs                        25.1.0
babel                        2.17.0
beautifulsoup4               4.13.3
bleach                       6.2.0
blinker                      1.4
cachetools                   5.5.1
certifi                      2025.1.31
cffi                         1.17.1
charset-normalizer           3.4.1
comm                         0.2.2
contourpy                    1.3.1
cryptography                 3.4.8
cycler                       0.12.

## Configure model to be generated as per requirement

In [20]:
import requests
import json

######## Set the model as per requirement
model_name = "whisper-base.en"          # whisper-tiny, whisper-tiny.en, whisper-base, whisper-base.en, whisper-small, whisper-small.en

######## Set the language, task, and options as per requirement
language_code = "<|en|>"             # <|en|>, <|fr|>, <|hi|>, <|ko|>, <|de|>, <|zh|>, <|ja|>, <|es|>, <|ar|>, <|ru|>, ...
task_code     = "<|transcribe|>"     # <|transcribe|>, <|translate|>
option_code   = "<|notimestamps|>"   # <|notimestamps|>, <|nocaptions|>

# URL of the JSON file which stores the code mappings
url = "https://huggingface.co/openai/whisper-large/resolve/main/added_tokens.json"

# Send a GET request to download the file
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON content
    code_mappings = response.json()
else:
    print(f"Failed to download the file. Status code: {response.status_code}")
    code_mappings = {}

# Construct forced_decoder_ids using the mappings
forced_decoder_ids = [
    [1, code_mappings[language_code]],
    [2, code_mappings[task_code]],
    [3, code_mappings[option_code]]
]

print(forced_decoder_ids)

[[1, 50259], [2, 50359], [3, 50363]]


##Import the libraries, load the model, do the inference

In [21]:
import tensorflow as tf
import transformers
import datasets

from datasets import load_dataset
from transformers import WhisperProcessor, WhisperFeatureExtractor, TFWhisperForConditionalGeneration, WhisperTokenizer

pretrained_model = f"openai/{model_name}"
tflite_model_path = f"{model_name}.tflite"
saved_model_dir = f"tf_{model_name}_saved"

feature_extractor = WhisperFeatureExtractor.from_pretrained(pretrained_model)
tokenizer = WhisperTokenizer.from_pretrained(pretrained_model, predict_timestamps=True)
processor = WhisperProcessor(feature_extractor, tokenizer)
model = TFWhisperForConditionalGeneration.from_pretrained(pretrained_model)

# Loading dataset
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
inputs = feature_extractor(ds[0]["audio"]["array"], sampling_rate=ds[0]["audio"]["sampling_rate"], return_tensors="tf")
input_features = inputs.input_features

# Generating Transcription
generated_ids = model.generate(input_features=input_features)
print(generated_ids)

transcription = processor.tokenizer.decode(generated_ids[0])
print(transcription)

# Save the model
# model.save(saved_model_dir) # not need to save here, saving using tf.saved_model.save() call

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

## Prompt fix, patch to make forced_decoder_ids work

In [ ]:
import tensorflow as tf
import numpy as np
from transformers import TFForceTokensLogitsProcessor, TFLogitsProcessor
from typing import List, Optional, Union, Any

# Patching methods of class TFForceTokensLogitsProcessor(TFLogitsProcessor):

def my__init__(self, force_token_map: List[List[int]]):
    force_token_map = dict(force_token_map)
    # Converts the dictionary of format {index: token} containing the tokens to be forced to an array, where the
    # index of the array corresponds to the index of the token to be forced, for XLA compatibility.
    # Indexes without forced tokens will have an negative value.
    force_token_array = np.ones((max(force_token_map.keys()) + 1), dtype=np.int32) * -1
    for index, token in force_token_map.items():
        if token is not None:
            force_token_array[index] = token
    self.force_token_array = tf.convert_to_tensor(force_token_array, dtype=tf.int32)

def my__call__(self, input_ids: tf.Tensor, scores: tf.Tensor, cur_len: int) -> tf.Tensor:
    def _force_token(generation_idx):
        batch_size = scores.shape[0]
        current_token = self.force_token_array[generation_idx]

        # Original code below generates NaN values when the model is exported to tflite
        # it just needs to be a negative number so that the forced token's value of 0 is the largest
        # so it will get chosen
        #new_scores = tf.ones_like(scores, dtype=scores.dtype) * -float("inf")
        new_scores = tf.ones_like(scores, dtype=scores.dtype) * -float(1)
        indices = tf.stack((tf.range(batch_size), tf.tile([current_token], [batch_size])), axis=1)
        updates = tf.zeros((batch_size,), dtype=scores.dtype)
        new_scores = tf.tensor_scatter_nd_update(new_scores, indices, updates)
        return new_scores

    scores = tf.cond(
        tf.greater_equal(cur_len, tf.shape(self.force_token_array)[0]),
        # If the current length is geq than the length of force_token_array, the processor does nothing.
        lambda: tf.identity(scores),
        # Otherwise, it may force a certain token.
        lambda: tf.cond(
            tf.greater_equal(self.force_token_array[cur_len], 0),
            # Only valid (positive) tokens are forced
            lambda: _force_token(cur_len),
            # Otherwise, the processor does nothing.
            lambda: scores,
        ),
    )
    return scores

TFForceTokensLogitsProcessor.__init__ = my__init__
TFForceTokensLogitsProcessor.__call__ = my__call__

##Define a model with a serving signature and save it in TF SavedModel format.















In [ ]:
class GenerateModel(tf.Module):
  def __init__(self, model):
    super(GenerateModel, self).__init__()
    self.model = model

  @tf.function(
    # shouldn't need static batch size, but throws exception without it (needs to be fixed)
    input_signature=[
      tf.TensorSpec((1, 80, 3000), tf.float32, name="input_features"),
    ],
  )
  def serving(self, input_features):
    outputs = self.model.generate(
      input_features,
      # change below if you think your output will be bigger
      # aka if you have bigger transcriptions
      # you can make it 200 for example
      max_new_tokens=448,
      return_dict_in_generate=True,
      forced_decoder_ids=forced_decoder_ids,
    )
    return {"sequences": outputs["sequences"]}

generate_model = GenerateModel(model=model)
tf.saved_model.save(generate_model, saved_model_dir, signatures={"serving_default": generate_model.serving})

## Convert the model from TF SavedModel format to TF lite

In [ ]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]

# Learn about post training quantization
# https://www.tensorflow.org/lite/performance/post_training_quantization

# Dynamic range quantization which reduces the size of the model to 25%
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Float16 quantization reduces the size to 50%
# converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

##Test tflite model using TFLite Interpreter. Check transcription for dataset



In [ ]:
# loaded model... now with generate!
interpreter = tf.lite.Interpreter(tflite_model_path)

tflite_generate = interpreter.get_signature_runner()
generated_ids = tflite_generate(input_features=input_features)["sequences"]
# print(generated_ids)

transcription = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
print(transcription)

<|startoftranscript|><|en|><|transcribe|><|notimestamps|> Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endofte

## Install faster-whisper for audio processing and testing model

In [ ]:
!git clone https://github.com/SYSTRAN/faster-whisper.git
!pip install faster-whisper

Cloning into 'faster-whisper'...
remote: Enumerating objects: 1105, done.
remote: Counting objects: 100% (501/501), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 1105 (delta 371), reused 279 (delta 279), pack-reused 604 (from 3)
Receiving objects: 100% (1105/1105), 38.07 MiB | 25.15 MiB/s, done.
Resolving deltas: 100% (697/697), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.1 MB/s eta 0:00:00


## Test all audio files in loop

In [ ]:
import os
import tensorflow as tf
from transformers import WhisperProcessor, WhisperFeatureExtractor
from faster_whisper import decode_audio

# Set up paths and model (whisper-tiny, whisper-tiny.en, whisper-base, whisper-base.en, whisper-small, whisper-small.en)
# model_name = "whisper-base.en"
# pretrained_model = f"openai/{model_name}"
# tflite_model_path = f"{model_name}.tflite"

######## NOTE: Specify the folder containing audio files
!git clone https://github.com/vilassn/audio_samples.git
audio_folder_path = 'audio_samples/en'
#audio_folder_path = '/content/drive/MyDrive/Colab Notebooks/audio'

feature_extractor = WhisperFeatureExtractor.from_pretrained(pretrained_model)
tokenizer = WhisperTokenizer.from_pretrained(pretrained_model, predict_timestamps=True)
processor = WhisperProcessor(feature_extractor, tokenizer)

interpreter = tf.lite.Interpreter(tflite_model_path)
tflite_generate = interpreter.get_signature_runner()

# Number of iterations you want the loop to run
iterations = 1000

for i in range(1, iterations + 1):  # Start from 1 to print iteration number
    print(f"Iteration {i}.......................................................\n")  # Print iteration number and newline

    # Loop through all files in the folder
    for audio_file_name in os.listdir(audio_folder_path):
        audio_file_path = os.path.join(audio_folder_path, audio_file_name)

        if audio_file_name.endswith('.wav'):  # Process only .wav files
            print(f"Processing {audio_file_name}...")

            # Preprocess the audio file
            input_audio = decode_audio(audio_file_path, sampling_rate=16000)
            input_features = feature_extractor(input_audio, sampling_rate=16000, return_tensors="tf").input_features

            # Run the model
            generated_ids = tflite_generate(input_features=input_features)["sequences"]

            # Decode and print transcription
            transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
            print(f"{transcription}\n")  # Add newline after each transcription

Cloning into 'audio_samples'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 20 (delta 0), reused 17 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 5.79 MiB | 21.02 MiB/s, done.
Iteration 1.......................................................

Processing english_test1.wav...
 Four score and seven years ago, our fathers brought forth on this continent a new nation, conceived in liberty and dedicated to the proposition that all men are created equal.

Processing jfk.wav...
 And so my fellow Americans, ask not what your country can do for you, ask what you can do for your country.

Processing english_test2.wav...
 This is the micro-machine, representing the most miniature modicator of Michael machine. Each one has dramatic details for a fixed-ridden precision-page art plus incredible micro-machine pocket placettes. There's a police station, fire station, restaurant,